In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import os
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import regularizers

In [2]:
sns.set_style("whitegrid")

In [3]:
dir_gdl = '../data/processed/2016-2019_3std_preprocessed.csv'
df = pd.read_csv(dir_gdl)
df_data = df[df['PARAM']=='PM10'].fillna(-1)
df_data.drop(columns=['Unnamed: 0'], inplace=True)
df_data = df_data[df_data.CEN != -1] #Elimina valores negativos en la columna de salida
print(df_data.shape)
df_data.head(5)

(27504, 13)


,FECHA,HORA,PARAM,AGU,ATM,CEN,LDO,LPIN,MIR,OBL,SFE,TLA,VAL
4,2016-01-01 00:00:00,00:00,PM10,49.92,146.95,86.12,174.04,-1.0,69.75,197.67,115.54,143.40,17.08
13,2016-01-01 01:00:00,01:00,PM10,52.80,-1.00,46.49,115.27,-1.0,68.99,138.09,84.24,100.46,29.15
22,2016-01-01 02:00:00,02:00,PM10,52.71,113.44,63.93,99.00,-1.0,117.70,98.79,135.39,82.05,30.89
31,2016-01-01 03:00:00,03:00,PM10,51.24,73.30,60.75,83.65,-1.0,160.30,97.94,117.60,114.74,38.74
40,2016-01-01 04:00:00,04:00,PM10,58.84,52.55,108.09,49.70,-1.0,180.89,134.39,164.68,118.83,51.48


In [4]:
X,Y = df_data[['AGU','ATM','LDO','LPIN','MIR','OBL','SFE','TLA','VAL']].to_numpy(), df_data["CEN"].to_numpy()   #separate data into input and output features

Y=np.reshape(Y, (-1,1))

X_std = (X - np.nanmin(np.where(X>=0, X, np.nan),axis=0)) / (X.max(axis=0) - np.nanmin(np.where(X>=0, X, np.nan),axis=0))
xscale = X_std * (1 - 0) + 0
xscale[X==-1]=-1

scaler_y = MinMaxScaler()
scaler_y.fit(Y)
yscale=scaler_y.transform(Y)

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size = 0.2) #split

## Testing code configurations

#### Creating layers

In [5]:
def neuron_layers(nx,nh,ny,hl,act,r, seed=None):
    
    tf.keras.regularizers.l1(l1=r)
    
    initializer = tf.keras.initializers.RandomNormal(seed=seed)
    
    model = Sequential()
    
    for i in range(1, 3+hl):
        
        if i == 1:
            model.add(Dense(nx, input_dim=9, kernel_initializer=initializer,
                            activation=act,kernel_regularizer='l1'))
            
        elif i == (2+hl):
            model.add(Dense(ny, activation='linear'))
            
        else:
            model.add(Dense(nh, activation=act))
            
    return model

## Previous model

In [8]:
nn = pd.read_csv('../output/data/RNA/NN_ModelsNotebook6c_v1.csv')
nn.rename(columns={'Unnamed: 0':'6c_number'}, inplace=True)
print(nn.shape)
nn.head(5)

(300, 18)


,6c_number,model_number,epochs,hidden_neurons,hidden_layers,activation,regularization,random_state_trainsplit,acc_train,mse_train,mae_train,rmse_train,ia_train,acc_test,mse_test,mae_test,rmse_test,ia_test
0,model1,model1209,200,100,3,relu,0.05,0,0.872036,84.137671,6.142833,9.172659,0.965,0.770278,154.232328,8.025615,12.419031,0.934
1,model2,model1209,200,100,3,relu,0.05,2,0.860820,89.543812,6.302811,9.462759,0.962,0.778719,161.088460,8.000407,12.692063,0.937
2,model3,model1209,200,100,3,relu,0.05,4,0.876833,81.290216,6.125698,9.016109,0.966,0.751454,164.426907,7.964257,12.822906,0.928
3,model4,model1209,200,100,3,relu,0.05,6,0.867700,87.738725,6.265878,9.366895,0.963,0.778086,143.987011,7.707175,11.999459,0.936
4,model5,model1209,200,100,3,relu,0.05,8,0.870076,85.908342,6.280808,9.268675,0.964,0.775326,147.526076,7.878047,12.146031,0.937


In [10]:
df_bestmodels = pd.read_csv('../output/data/RNA/BestModels_Notebook6c.csv')
print(df_bestmodels.shape)
df_bestmodels.head(4)

(10, 5)


,Unnamed: 0,model,6b_model,variable,result
0,0,model59,model1254,mse_train,69.556236
1,1,model59,model1254,mae_train,5.496591
2,2,model59,model1254,rmse_train,8.340038
3,3,model255,model788,mse_test,126.002045


## Testing for different seeds in weights

In [24]:
i = 1
bm = list(df_bestmodels.model.unique())

models = {}

rndm = range(0,100,2)

for mn in bm:
    nh = int(nn.loc[nn['6c_number']==mn,'hidden_neurons'])
    hl = int(nn.loc[nn['6c_number']==mn,'hidden_layers'])
    e = int(nn.loc[nn['6c_number']==mn,'epochs'])
    a = 'relu'
    r = float(nn.loc[nn['6c_number']==mn,'regularization'])
    rnd_st = int(nn.loc[nn['6c_number']==mn,'random_state_trainsplit'])
    N6b = nn.loc[nn['6c_number']==mn,'model_number'].values[0]

    for rnd_init in rndm:

        X_train,X_test,Y_train,Y_test = train_test_split(X, Y, 
                                                         test_size = 0.2, 
                                                         random_state=rnd_st) #split

        model = neuron_layers(10,nh,1,hl,a,r,seed=rnd_init)

        model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

        history = model.fit(X_train, Y_train, epochs=e, 
                            batch_size=50,  verbose=0, validation_split=0.2)

        #statistics for train
        y_hat= model.predict(X_train)
        acc_train = r2_score(Y_train, y_hat)
        mse_train = mean_squared_error(Y_train, y_hat)
        mae_train = mean_absolute_error(Y_train, y_hat)
        rmse_train = mean_squared_error(Y_train, y_hat, squared=False)

        num_train = ((y_hat - Y_train)**2).sum()
        den_train = ((abs(y_hat - Y_train.mean()) + 
                abs(Y_train - Y_train.mean()))**2).sum()
        ia_train = round(1 - (num_train / den_train),3)

        #statistics for test
        y_hat = model.predict(X_test)
        acc_test = r2_score(Y_test, y_hat)
        mse_test = mean_squared_error(Y_test, y_hat)
        mae_test = mean_absolute_error(Y_test, y_hat)
        rmse_test = mean_squared_error(Y_test, y_hat, squared=False)

        num_test = ((y_hat - Y_test)**2).sum()
        den_test = ((abs(y_hat - Y_test.mean()) + 
                abs(Y_test - Y_test.mean()))**2).sum()
        ia_test = round(1 - (num_test / den_test),3)

        models['model'+str(i)] = [mn, N6b, e, nh, hl, a, r, rnd_st,rnd_init,
                                               acc_train, mse_train, mae_train, rmse_train, ia_train,
                                               acc_test, mse_test, mae_test, rmse_test, ia_test]

        print ('\n*For model',str(i),'settings are:','-epochs:',str(e),'-hidden neurons:',str(nh),'-hidden layers:',str(hl),'-activation:',a,
               '-regularization cost:',r,'-random_state:',rnd_st,'-weight_init:',rnd_init,
               '\nAccuracy for training is:', str(acc_train),'Accuracy for test is:',str(acc_test))

        i += 1


*For model 1 settings are: -epochs: 200 -hidden neurons: 100 -hidden layers: 10 -activation: relu -regularization cost: 1.0 -random_state: 16 -weight_init: 0 
Accuracy for training is: 0.8609602633292073 Accuracy for test is: 0.7227261303244574

*For model 2 settings are: -epochs: 200 -hidden neurons: 100 -hidden layers: 10 -activation: relu -regularization cost: 1.0 -random_state: 16 -weight_init: 2 
Accuracy for training is: 0.8762378582304575 Accuracy for test is: 0.7577470901124966

*For model 3 settings are: -epochs: 200 -hidden neurons: 100 -hidden layers: 10 -activation: relu -regularization cost: 1.0 -random_state: 16 -weight_init: 4 
Accuracy for training is: 0.8742058826337504 Accuracy for test is: 0.7458550029104958

*For model 4 settings are: -epochs: 200 -hidden neurons: 100 -hidden layers: 10 -activation: relu -regularization cost: 1.0 -random_state: 16 -weight_init: 6 
Accuracy for training is: 0.8775320191223535 Accuracy for test is: 0.7573225652795198

*For model 5 se

In [25]:
df_models = pd.DataFrame(models)
df_models.head(5)

,model1,model2,model3,model4,model5,model6,model7,model8,model9,model10,...,model191,model192,model193,model194,model195,model196,model197,model198,model199,model200
0,model59,model59,model59,model59,model59,model59,model59,model59,model59,model59,...,model149,model149,model149,model149,model149,model149,model149,model149,model149,model149
1,model1254,model1254,model1254,model1254,model1254,model1254,model1254,model1254,model1254,model1254,...,model732,model732,model732,model732,model732,model732,model732,model732,model732,model732
2,200,200,200,200,200,200,200,200,200,200,...,100,100,100,100,100,100,100,100,100,100
3,100,100,100,100,100,100,100,100,100,100,...,75,75,75,75,75,75,75,75,75,75
4,10,10,10,10,10,10,10,10,10,10,...,5,5,5,5,5,5,5,5,5,5


In [26]:
df_models.to_csv('../output/data/RNA/NN_RawModels_Notebooks6d_v1.csv')

In [27]:
df_models = df_models.transpose()
df_models = df_models.rename(columns={0:'6c_model',1:'6b_model', 2:'epochs',3:'hidden_neurons',4:'hidden_layers',5:'activation',
                        6:'regularization',7:'random_state_trainsplit',8:'random_WeightInit',
                        9:'acc_train',10:'mse_train',11:'mae_train',12:'rmse_train',13:'ia_train',
                        14:'acc_test',15:'mse_test',16:'mae_test',17:'rmse_test',18:'ia_test'})
print(df_models.shape)
df_models.head(5)

(200, 19)


,6c_model,6b_model,epochs,hidden_neurons,hidden_layers,activation,regularization,random_state_trainsplit,random_WeightInit,acc_train,mse_train,mae_train,rmse_train,ia_train,acc_test,mse_test,mae_test,rmse_test,ia_test
model1,model59,model1254,200,100,10,relu,1.0,16,0,0.86096,91.913893,6.459054,9.587173,0.965,0.722726,182.254045,8.366229,13.50015,0.926
model2,model59,model1254,200,100,10,relu,1.0,16,2,0.876238,81.814455,5.998837,9.045134,0.966,0.757747,159.234524,7.979057,12.618816,0.931
model3,model59,model1254,200,100,10,relu,1.0,16,4,0.874206,83.157717,6.094639,9.119085,0.968,0.745855,167.051276,8.194086,12.924832,0.932
model4,model59,model1254,200,100,10,relu,1.0,16,6,0.877532,80.958934,5.99864,8.997718,0.966,0.757323,159.513567,7.863795,12.629868,0.929
model5,model59,model1254,200,100,10,relu,1.0,16,8,0.878634,80.230342,5.976462,8.957139,0.966,0.735039,174.160898,8.199043,13.197003,0.921


In [28]:
df_models.to_csv('../output/data/RNA/NN_ModelsNotebook6d_v1.csv')

## Filtering best models

In [29]:
df_best_models = pd.DataFrame(columns=['model','6b_model','6c_model','variable','result'])

min_cols = ['mse_train','mae_train','rmse_train',
       'mse_test','mae_test','rmse_test']
max_cols = ['acc_train','ia_train',
            'acc_test','ia_test']

i = 0
for c in min_cols:
    val = df_models[c].min()
    df_tmp = df_models.loc[df_models[c]==val].copy()
    df_tmp[['epochs']] = df_tmp[['epochs']]/200
    df_tmp[['hidden_neurons']] = df_tmp[['hidden_neurons']]/100
    df_tmp[['hidden_layers']] = df_tmp[['hidden_layers']]/10
    df_tmp[['regularization']] = df_tmp[['regularization']]/1
    model = df_tmp[['epochs','hidden_neurons',
                    'hidden_layers','regularization']].sum(axis=1).idxmin()
    N6b = df_tmp.loc[df_tmp.index==model,'6b_model']
    N6c = df_tmp.loc[df_tmp.index==model,'6c_model']
    df_best_models.loc[i] = [model, N6b[0],N6c[0], c, val]
    i += 1
    
for c in max_cols:
    val = df_models[c].max()
    df_tmp = df_models.loc[df_models[c]==val].copy()
    df_tmp[['epochs']] = df_tmp[['epochs']]/200
    df_tmp[['hidden_neurons']] = df_tmp[['hidden_neurons']]/100
    df_tmp[['hidden_layers']] = df_tmp[['hidden_layers']]/10
    df_tmp[['regularization']] = df_tmp[['regularization']]/1
    model = df_tmp[['epochs','hidden_neurons',
                    'hidden_layers','regularization']].sum(axis=1).idxmin()
    N6b = df_tmp.loc[df_tmp.index==model,'6b_model']
    N6c = df_tmp.loc[df_tmp.index==model,'6c_model']
    df_best_models.loc[i] = [model, N6b[0],N6c[0], c, val]
    i += 1

In [30]:
df_best_models

,model,6b_model,6c_model,variable,result
0,model9,model1254,model59,mse_train,71.800892
1,model9,model1254,model59,mae_train,5.612536
2,model9,model1254,model59,rmse_train,8.473541
3,model130,model732,model111,mse_test,125.692934
4,model130,model732,model111,mae_test,7.367142
5,model130,model732,model111,rmse_test,11.211286
6,model9,model1254,model59,acc_train,0.891386
7,model9,model1254,model59,ia_train,0.970000
8,model187,model732,model149,acc_test,0.808445
9,model74,model788,model255,ia_test,0.948000


In [31]:
df_best_models.to_csv('../output/data/RNA/BestModels_Notebook6d.csv')

In [33]:
model_var = ['random_WeightInit']

for m in model_var:
    df_models.groupby([m]).agg(['mean','std']).to_csv(f'../output/data/RNA/{m}_ModelAnalysis_Notebook6d.csv')

In [ ]:
df_models.acc_train.idxmax()
df_models.mse_train.idxmin()
df_models.mae_train.idxmin()
df_models.rmse_train.idxmin()
df_models.std_res_train.idxmin()
df_models.ia_train.idxmax()

In [ ]:
df_models.acc_test.idxmax()
df_models.mse_test.idxmin()
df_models.mae_test.idxmin()
df_models.rmse_test.idxmin()
df_models.std_res_test.idxmin()
df_models.ia_test.idxmax()